In [13]:
import pandas as pd
import pickle
import os

import ipywidgets as widgets
from ipywidgets import interact

from src.data_load import load_tables, load_instance, load_distances
from src.filtering import filter_labors_by_city, filter_labors_by_date, flexible_filter
from src.plotting import plot_metrics_comparison, plot_gantt_labors_by_driver, plot_results
from src.metrics import collect_vt_metrics_range, show_day_report_dayonly, compute_indicators, collect_results_to_df, \
    compute_metrics_with_moves
from src.config import *
from src.experimentation_config import fechas_dict, SALARY, EXTRA_TIME
from src.alpha_tuning_utils import collect_alpha_results_to_df, filter_df_on_hyperparameter_selection, include_all_city, \
    compute_algo_vs_historic_metrics, aggregate_by_city, aggregate_by_metric, \
    alphas, metrics, iterations_nums

data_path = '../data'
# instance = 'inst4b'
instance = 'instr1'

fechas = fechas_dict[instance]
dist_method = 'haversine'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df = load_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, 'osrm', instance)

# Description

This notebook provides an detailed analysis of the experimentation of alpha calibration with the three different objectives for the calibration. Providing detail view on day-to-day metrics of the assignment and scheduling.

- hybrid
- extra_time
- driver_move_distance

# Upload and process results data

In [14]:
labors_hist_df, moves_hist_df = collect_results_to_df(data_path, instance, fechas, assignment_type='historic')
labors_algo_df, moves_algo_df = collect_alpha_results_to_df(data_path, instance, dist_method, metrics, alphas, iterations_nums)

hyperparameter_selection = {
    'driver_distance': {'num_iter':500, 'alpha':0},
    'driver_extra_time':    {'num_iter':500, 'alpha':0.1},
    'hybrid':               {'num_iter':500, 'alpha':0.1}
}

labors_algo_df = filter_df_on_hyperparameter_selection(labors_algo_df, hyperparameter_selection)
moves_algo_df = filter_df_on_hyperparameter_selection(moves_algo_df, hyperparameter_selection)

labors_hist_df = include_all_city(labors_hist_df)
moves_hist_df = include_all_city(moves_hist_df)


# Results

In [15]:
metricas = ['vt_count', 'num_drivers', "labores_por_conductor", 'total_distance', 
            'labor_extra_time', 'driver_extra_time', 'driver_move_distance']
# metricas = ['driver_move_distance']


def plot_metrics_comparison_interactive(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_df: pd.DataFrame,
    moves_algo_df: pd.DataFrame,
    cities: list[str],
    metrics: list[str], 
    metricas,
    dist_dict: dict,
    fechas: tuple[str, str]
):
    """
    Interactive version of plot_metrics_comparison with ipywidgets.
    Allows selecting the city dynamically from a dropdown.
    """

    def _plot_for_city(city: str, 
                       metric,
                       ):

        alphas_selection = {'hybrid':0.1,
                            'driver_extra_time':0.1, 
                            'driver_distance':0}
        alpha = alphas_selection[metric]
        num_iter=500

        labors_algo_temp = labors_algo_df[(labors_algo_df['metric']==metric) &
                       (labors_algo_df['alpha']==alpha) &
                       (labors_algo_df['num_iter']==num_iter)]
        moves_algo_temp = moves_algo_df[(moves_algo_df['metric']==metric) &
                       (moves_algo_df['alpha']==alpha) &
                       (moves_algo_df['num_iter']==num_iter)]
        
        

        bbb = labors_algo_temp.copy()
        bbb['city'] = 'ALL'
        labors_algo_temp = pd.concat([labors_algo_temp, bbb])

        
        ddd = moves_algo_temp.copy()
        ddd['city'] = 'ALL'
        moves_algo_temp = pd.concat([moves_algo_temp, ddd])
        
        
        # Call your existing function
        figs = plot_metrics_comparison(
            labors_hist_df=labors_hist_df,
            moves_hist_df=moves_hist_df,
            labors_algo_df=labors_algo_temp,
            moves_algo_df=moves_algo_temp,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas, 
            # group_by=['alpha', 'num_iterations', 'metric']
        )
        # return figs

    # Create dropdown
    city_dropdown = widgets.Dropdown(
        options=['ALL'] + cities,
        value=cities[0],  # default city
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    metrics_dropdown = widgets.Dropdown(
        options=metrics,
        value=metrics[0],  # default city
        description="Métrica:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # Link dropdown with plotting function
    interact(_plot_for_city, city=city_dropdown, metric=metrics_dropdown)

plot_metrics_comparison_interactive(
    labors_hist_df,
    moves_hist_df,
    labors_algo_df,
    moves_algo_df,
    cities=valid_cities,  # put the city codes here
    metrics=metrics, 
    # alphas=alphas,
    # iterations_nums=iterations_nums,
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=(fechas[0], fechas[-1])
)

interactive(children=(Dropdown(description='Ciudad:', index=1, layout=Layout(width='300px'), options=('ALL', '…

# KPI's agregados

In [16]:
comparison = compute_algo_vs_historic_metrics(
    labors_algo_df, moves_algo_df,
    labors_hist_df, moves_hist_df,
    fechas, dist_dict, valid_cities, metrics,
    hyperparameter_selection,
    dist_method="haversine"
)

comparison_city = aggregate_by_city(comparison, SALARY, EXTRA_TIME)
comparison_metric = aggregate_by_metric(comparison, SALARY, EXTRA_TIME)

comparison_metric

,metric,vt_count,num_drivers,labor_extra_time,driver_extra_time,driver_move_distance,vt_count_pct,num_drivers_pct,labor_extra_time_pct,driver_extra_time_pct,driver_move_distance_pct,driver_extra_time_h,salary_cost,extra_time_cost,total,total_con_ajuste,total_con_ajuste_mes,total_con_ajuste_anio
0,driver_distance,-12,18,-2312.9,1256.4,2707.930034,0.718066,7.657271,-28.80205,6.75271,49.596598,20.94,854100,31050.0,885150.0,752378.0,3009512.0,36114144.0
1,driver_extra_time,-12,19,-2064.6,2388.0,2630.087795,0.718066,7.965913,-26.303765,10.351248,48.824837,39.80,901550,59016.0,960566.0,816481.0,3265924.0,39191088.0
2,hybrid,-12,17,-2102.0,2526.0,2617.781330,0.718066,7.802515,-26.630149,10.998881,49.056261,42.10,806650,62426.0,869076.0,738715.0,2954860.0,35458320.0


In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_comparison_metrics(comparison_metric, indicators, colors):
    """
    Plots absolute and percentage gaps for multiple indicators,
    row by row (absolute left, percentage right).

    Parameters
    ----------
    comparison_metric : pd.DataFrame
        Dataframe with absolute and percentage gaps per optimization objective.
    indicators : list of str
        List of indicators to plot (columns in comparison_metric).
    colors : dict
        Mapping from optimization objective -> color.
    """
    figs = []
    for col in indicators:
        abs_col = col
        pct_col = f"{col}_pct"

        # Create subplot row: absolute (left), perceptual (right)
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=(f"{col} – Absolute gap", f"{col} – Perceptual gap")
        )

        # --- Absolute gap ---
        for obj in comparison_metric["metric"].unique():
            subset = comparison_metric[comparison_metric["metric"] == obj]
            if not subset.empty:
                y_vals = pd.to_numeric(subset[abs_col], errors="coerce").fillna(0)
                fig.add_trace(
                    go.Bar(
                        x=[obj],
                        y=y_vals,
                        name=obj,
                        marker_color=colors.get(obj, None),
                        text=y_vals.round(1).astype(str),
                        textposition="auto"
                    ),
                    row=1, col=1
                )

        # --- Perceptual gap ---
        if pct_col in comparison_metric.columns:
            for obj in comparison_metric["metric"].unique():
                subset = comparison_metric[comparison_metric["metric"] == obj]
                if not subset.empty:
                    y_vals = pd.to_numeric(subset[pct_col], errors="coerce").fillna(0)
                    text_vals = y_vals.round(1).astype(str) + "%"
                    fig.add_trace(
                        go.Bar(
                            x=[obj],
                            y=y_vals,
                            name=obj,
                            marker_color=colors.get(obj, None),
                            text=text_vals,
                            textposition="auto"
                        ),
                        row=1, col=2
                    )

        # Layout per row
        fig.update_layout(
            title_text=f"Comparison for {col}",
            barmode="group",
            legend=dict(orientation="h", y=-0.2),  # legend just below row
            height=400,
            width=900
        )
        figs.append(fig)

    return figs


In [18]:
indicators = ["vt_count", "num_drivers", 
              "driver_extra_time", "driver_move_distance"]

colors = {
    "driver_distance": "#4C78A8",   # muted blue
    "driver_extra_time": "#F58518", # warm orange
    "hybrid": "#54A24B",            # fresh green
}


figs = plot_comparison_metrics(comparison_metric, indicators, colors)

for fig in figs:
    fig.show()
